<a href="https://colab.research.google.com/github/ciccio1982/Colab/blob/main/servizio_in_ricezione_con_ngork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print("Installazione delle librerie necessarie: fastapi, uvicorn, ngrok, loguru, nest_asyncio...")
!pip install fastapi uvicorn ngrok loguru nest_asyncio pyngrok

print("Installazione completata. Ora puoi eseguire la cella successiva con il codice dell'applicazione.")

Installazione delle librerie necessarie: fastapi, uvicorn, ngrok, loguru, nest_asyncio...
Installazione completata. Ora puoi eseguire la cella successiva con il codice dell'applicazione.


In [ ]:
import os
from contextlib import asynccontextmanager
import ngrok as ngrok_sdk # Importa la libreria ngrok Python SDK
import uvicorn
from fastapi import FastAPI
from loguru import logger
import nest_asyncio
import pyngrok
import threading # Importa il modulo threading
import time # Importa il modulo time per sleep
import socket # Importa socket per gestire errori di porta
import logging

# Applica nest_asyncio per permettere l'esecuzione di Uvicorn in un ambiente come Colab
nest_asyncio.apply() # Spostato qui per assicurare che venga applicato il prima possibile

# 2. Configura logging FORZATO visibile
logging.basicConfig(level=logging.INFO, force=True)
logger_uvicorn = logging.getLogger("uvicorn")
logger_uvicorn.setLevel(logging.INFO)

# Porta su cui girerà l'applicazione FastAPI
APPLICATION_PORT = 8001 # Cambiata la porta da 8000 a 8001

# Recupera l'authtoken ngrok dai "Secrets" di Colab
# È fondamentale avere l'authtoken configurato correttamente per ngrok
# Assicurati di aver chiamato il segreto esattamente 'NGROK_AUTHTOKEN' in Colab Secrets.
NGROK_AUTHTOKEN = "384dhlb1DJ1x08HU1t8VUGy0sUW_3cFaNVzy7WshJrKRQyjAM"


if not NGROK_AUTHTOKEN:
    print("\nATTENZIONE: Authtoken ngrok non trovato nelle variabili d'ambiente (Secrets di Colab).")
    print("Per esporre il tuo servizio FastAPI pubblicamente con ngrok, è necessario un authtoken.")
    print("1. Vai su https://ngrok.com/signup per creare un account.")
    print("2. Ottieni il tuo authtoken dalla dashboard (https://dashboard.ngrok.com/get-started/your-authtoken).\n   Assicurati che sia quello per 'ngrok v3' o successivo.")
    print("3. In Colab, clicca sull'icona della chiave (Secrets) a sinistra.")
    print("4. Aggiungi un nuovo segreto con il nome 'NGROK_AUTHTOKEN' e incolla il tuo authtoken come valore.")
    print("5. Esegui nuovamente questa cella dopo aver aggiunto il segreto.\n")

@asynccontextmanager
async def lifespan(app: FastAPI):
    public_url = None
    if NGROK_AUTHTOKEN:
        logger.info("Avvio ngrok...")
        try:
            ngrok_sdk.set_auth_token(NGROK_AUTHTOKEN)
            # Termina qualsiasi processo ngrok esistente per evitare conflitti
            print("Terminazione di eventuali processi ngrok precedenti...")
            ngrok_sdk.disconnect()
            ngrok_sdk.kill()

            # Avvia il tunnel ngrok
            tunnel = await ngrok_sdk.connect(APPLICATION_PORT)
            public_url = tunnel.url()
            logger.info(f"URL PUBBLICO: {public_url}")
            print(f"\n🎉 **Il tuo servizio è online!** Puoi richiamarlo qui: {public_url}\n")
            print(f"Endpoint di esempio: {public_url}/test/ColabUser")
            print(f"Endpoint di esempio: {public_url}/webhook (metodo POST)")
        except Exception as e:
            logger.error(f"Errore durante l'avvio del tunnel ngrok: {e}")
            print(f"\nErrore durante l'avvio del tunnel ngrok: {e}")
            print("Assicurati che il tuo authtoken ngrok sia valido e correttamente configurato nei Secrets di Colab.")
    else:
        print("Avviso: Authtoken ngrok non configurato. Il tunnel ngrok non verrà avviato. Il servizio sarà disponibile solo localmente.")

    yield

    if NGROK_AUTHTOKEN:
        logger.info("Chiusura ngrok...")
        ngrok_sdk.disconnect()
        ngrok_sdk.kill() # Assicura che tutti i processi ngrok siano terminati

app = FastAPI(lifespan=lifespan)

@app.get("/")
async def root():
    print("🌟 ROOT CHIAMATO! Visibile subito.", flush=True)
    logger_uvicorn.info("QUALCOSA:")
    logger.info(f"Payload ricevuto")
    return {"messaggio": "bravo!", "status": "OK", "info": "Visita /test/TuoNome o fai un POST a /webhook"}

@app.get("/test/{nome}")
async def test(nome: str):
    print("🌟 ROOT CHIAMATO! Visibile subito.", flush=True)
    logger_uvicorn.info("QUALCOSA:")
    logger.info(f"Payload ricevuto")
    print("fatto ", flush=True)
    return {"ciao": nome, "da": "ngrok", "servizio": "FastAPI"}

@app.post("/webhook")
async def webhook(payload: dict):
    print("fatto ", flush=True)
    logger.info(f"Payload ricevuto: {payload}")
    return {"ricevuto": True, "data": payload, "messaggio": "Dati JSON ricevuti con successo!"}

print("Avvio del server FastAPI con uvicorn in un thread separato...")

def run_uvicorn():
    try:
        # Usa host="0.0.0.0" per rendere il server accessibile all'interno del container Colab
        # Rimosso loop='asyncio' per compatibilità con nest_asyncio in ambienti come Colab
        uvicorn.run(app, host="0.0.0.0", port=APPLICATION_PORT)
    except socket.error as e:
        if e.errno == 98: # Errno 98 is 'Address already in use'
            print(f"\nERRORE: La porta {APPLICATION_PORT} è già in uso. Potrebbe esserci un altro processo in esecuzione.")
            print("Ti consiglio di RIavviare il tuo ambiente di runtime Colab (Runtime > Restart runtime...) e ri-eseguire tutte le celle.\n")
            print("In alternativa, puoi provare a cambiare il valore di APPLICATION_PORT (es. da 8000 a 8001) in questa cella e ri-eseguire.")
        else:
            print(f"\nERRORE durante l'avvio di Uvicorn: {e}")

# Esegue l'applicazione FastAPI in un thread separato.
# Questo permette alla cella di continuare l'esecuzione dopo l'avvio del server.
uvicorn_thread = threading.Thread(target=run_uvicorn)
uvicorn_thread.start()

# Dai un po' di tempo al server per avviarsi prima che la cella termini (opzionale)
#time.sleep(5)
#print("Server Uvicorn avviato in background. Controlla gli output precedenti per l'URL ngrok (se configurato).")

# Sostituisci time.sleep(5) e print finale con QUESTO:
print("🚀 Server avviato! MANTIENI QUESTA CELLA ATTIVA...")
print("Log attivi ogni 30s per confermare vivo 👇")

import time
start_time = time.time()

# LOOP INFINITO per tenere cella attiva + heartbeat
while True:
    elapsed = time.time() - start_time
    print(f"⏰ Vivo da {elapsed/60:.1f} min | URL: {public_url if 'public_url' in locals() else 'ngrok non partito'}", flush=True)
    time.sleep(30)  # Print ogni 30s, non blocca


Avvio del server FastAPI con uvicorn in un thread separato...
🚀 Server avviato! MANTIENI QUESTA CELLA ATTIVA...
Log attivi ogni 30s per confermare vivo 👇
⏰ Vivo da 0.0 min | URL: ngrok non partito


INFO:     Started server process [301]
INFO:     Waiting for application startup.
2026-01-11 15:15:33.859 | INFO     | __main__:lifespan:44 - Avvio ngrok...


Terminazione di eventuali processi ngrok precedenti...


INFO:ngrok.session:Session created without auth token
INFO:ngrok.listener:Created listener "a821c710fe084d4a118dbaf64990819e" with url "https://hoglike-sidney-uncatastrophic.ngrok-free.dev"
2026-01-11 15:15:34.377 | INFO     | __main__:lifespan:55 - URL PUBBLICO: https://hoglike-sidney-uncatastrophic.ngrok-free.dev
INFO:ngrok.listener:Listener "a821c710fe084d4a118dbaf64990819e" forwarding to "tcp://localhost:8001"
INFO:     Application startup complete.
INFO:ngrok.tunnel_ext:forward_tunnel; tunnel_id="a821c710fe084d4a118dbaf64990819e" url=tcp://localhost:8001
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)



🎉 **Il tuo servizio è online!** Puoi richiamarlo qui: https://hoglike-sidney-uncatastrophic.ngrok-free.dev

Endpoint di esempio: https://hoglike-sidney-uncatastrophic.ngrok-free.dev/test/ColabUser
Endpoint di esempio: https://hoglike-sidney-uncatastrophic.ngrok-free.dev/webhook (metodo POST)
INFO:     2001:b07:6478:8e17:b919:e4e5:23c3:15dd:0 - "GET /webhook HTTP/1.1" 405 Method Not Allowed
INFO:     2001:b07:6478:8e17:b919:e4e5:23c3:15dd:0 - "GET /webhook HTTP/1.1" 405 Method Not Allowed
INFO:     2001:b07:6478:8e17:b919:e4e5:23c3:15dd:0 - "GET /webhook HTTP/1.1" 405 Method Not Allowed
⏰ Vivo da 0.5 min | URL: ngrok non partito
🌟 ROOT CHIAMATO! Visibile subito.


INFO:     QUALCOSA:
2026-01-11 15:16:05.942 | INFO     | __main__:root:79 - Payload ricevuto


INFO:     2001:b07:6478:8e17:b919:e4e5:23c3:15dd:0 - "GET / HTTP/1.1" 200 OK
⏰ Vivo da 1.0 min | URL: ngrok non partito
⏰ Vivo da 1.5 min | URL: ngrok non partito
⏰ Vivo da 2.0 min | URL: ngrok non partito
⏰ Vivo da 2.5 min | URL: ngrok non partito
⏰ Vivo da 3.0 min | URL: ngrok non partito
⏰ Vivo da 3.5 min | URL: ngrok non partito
⏰ Vivo da 4.0 min | URL: ngrok non partito
⏰ Vivo da 4.5 min | URL: ngrok non partito
⏰ Vivo da 5.0 min | URL: ngrok non partito
⏰ Vivo da 5.5 min | URL: ngrok non partito
⏰ Vivo da 6.0 min | URL: ngrok non partito


KeyboardInterrupt: 